https://aeda.tistory.com/16 


In [ ]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 970 kB/s 
     |████████████████████████████████| 465 kB 58.8 MB/s 


In [ ]:
book = pd.read_csv('/content/drive/Shareddrives/2022-2 KUBIG 추천시스템/2팀/6주차/도서 추천시스템/Movie_DB_added2.csv', encoding='cp949') #한국어 포함된 데이터

In [ ]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2244 entries, 0 to 2243
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Column1  2244 non-null   int64  
 1   제목       2244 non-null   object 
 2   원제       2244 non-null   object 
 3   제작연도     2243 non-null   float64
 4   국가       2244 non-null   object 
 5   장르       2215 non-null   object 
 6   상영시간     2244 non-null   object 
 7   연령등급     2145 non-null   object 
 8   내용       2244 non-null   object 
 9   리뷰       2244 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 175.4+ KB


In [ ]:
book.head(10)

,Column1,제목,원제,제작연도,국가,장르,상영시간,연령등급,내용,리뷰
0,0,물방울을 그리는 남자,L'homme qui peint des gouttes d'eau,2020.0,"프랑스,한국",다큐멘터리,1시간 19분,전체 관람가,50년간 묵묵히 '물방울'만을 그리며?물방울 작가로 사랑받은 화가 김창열. 침묵과 ...,"[[['이동진 평론가', '4.0', '삶이 어떻게 영화가 될 수 있는지 매우 아름..."
1,1,2차 송환,2차 송환,2022.0,한국,다큐멘터리,2시간 36분,12세 관람가,"""한국전쟁은 아직 끝나지 않았다!""?\n\n현재까지 남과 북은 정치 공작원들을 상호...","[[['이동진 평론가', '3.5', '여전히 뜻을 꺾지 않는 사람들을 다시 21년..."
2,2,달이 지는 밤,달이 지는 밤,2020.0,한국,모험,1시간 10분,15세 관람가,중년의 여인이 무주 시외버스 터미널에서 내린다. 그녀는 마을길을 지나 숲으로 들어간...,"[[['이동진 평론가', '3.5', '꿈인 듯 현실인 듯 죽음인 듯 삶인 듯, 경..."
3,3,썬더버드,썬더버드,2021.0,한국,범죄/액션,1시간 37분,15세 관람가,돈이 미치게 필요한 태균\n돈이 든 자동차를 잃어버린 태민\n돈은 중요하지 않은 미...,"[[['이동진 평론가', '3.0', '사북의 차가운 밤거리에서 실제로 엿보는 것 ..."
4,4,기기묘묘,기기묘묘,2022.0,한국,공포/액션/스릴러,1시간 56분,15세 관람가,"땅을 둘러 싼 이웃들의 다툼,?엄마와 딸의 기이한 관계,?낙향한 청년과 괴인의 기묘...","[[['이동진 평론가', '2.5', '이탁 오민애 김재화. <불모지>가 참 좋았다..."
5,5,9명의 번역가,Les Traducteurs,2019.0,"프랑스,벨기에",미스터리/스릴러,1시간 45분,15세 관람가,밀실에서 유출된 세계적 베스트셀러..!\n범인은 우리들 중에 있다!\n\n화제의 베...,"[[['이동진 평론가', '3.5', '효과적인 트릭과 반전들 아래, 승부에서 이기..."
6,6,홈리스,홈리스,2020.0,한국,드라마/가족,1시간 23분,12세 관람가,"이사를 앞둔 어린 부부 ‘한결’과 ‘고운’, 하지만 설렘도 잠시, 보증금 사기를 당...","[[['이동진 평론가', '3.0', '영화가 미뤄둔 그 이후의 일들을 떠올리면 더..."
7,7,라이트하우스,The Lighthouse,2019.0,"미국,브라질,캐나다",드라마/판타지/역사/공포/미스터리/스릴러,1시간 49분,청소년 관람불가,"1890년대, 동떨어진 섬의 등대를 지키게 된 두 등대지기의 이야기. 고독과 외로움...","[[['혁준', '4.0', '구원의 빛은 너무나 밝고 강렬해서\n이미 시커먼 어둠..."
8,8,더 위치,The VVitch: A New-England Folktale,2015.0,"미국,영국,캐나다,브라질",드라마/공포/미스터리,1시간 32분,15세 관람가,"1692년 살렘마녀재판을 소재로, 일련의 기이한 사건들로 광기에 사로잡히는 가족의 ...","[[['윤종원', '5.0', '2015 년 선댄스 드라마 부문 감독상의 영광을 누..."
9,9,성적표의 김민영,성적표의 김민영,2021.0,한국,드라마,1시간 34분,전체 관람가,"“앞으로 무엇을 하든, 그때의 우리 같았으면 좋겠어”\n\n기숙사 생활을 하며 삼행...","[[['이동진 평론가', '3.5', '만물이 정지한 순간에 끼어들어 오는 딸꾹질 ..."


책 정보 데이터 변환

lightfm 사이트에서는 countvectorizeration을 사용했는데, 이는 한국어의 처리를 하는데에 적합하지 않다. 그렇기 때문에 한국어는 불용어를 처리하여 BoW 방식으로 처리해줘야한다. 


In [ ]:
#품사태깅
#특수 문자 제거 + 숫자(??)
#문장부호 제거
#영어, 한자 제거 
#불용어 제거 

In [ ]:
from konlpy.tag import Okt
import re #정규식

In [ ]:
#품사 태깅

okt = Okt() # 형태소 분석기

def clean1(text):
   clean = []

   for word in okt.pos(text, stem = True): #어간 추출
    if word[1] not in ["Josa", "Eomi", "Punctuation"]:
      clean.append(word[0])
    
   return " ".join(clean)

In [ ]:
book.내용 = book.내용.apply(lambda x : clean1(x)) 

In [ ]:
print(book.내용)

0       50년 간 묵묵하다 물방울 그리다 물방울 작가 사랑받다 화가 김창열 침묵 고독 가득...
1       한국 전쟁 아직 끝나다 않다 \n\n 현재 남 북 정치 공작원 들 상호 침투 시키다...
2       중년 여인 무주 시외버스 터미널 내 린다 그녀 마을 길 지나 숲 들어가다 숲 어디 ...
3       돈 미치다 필요하다 태균 \n 돈 들다 자동차 잃어버리다 태민 \n 돈 중요하다 않...
4       땅 두르다 싸다 이웃 들 다툼 엄마 딸 기이하다 관계 낙향 청년 괴인 기묘하다 만남...
                              ...                        
2239    제 2 차 세계대전 발발 전 나치 점령 독일 연극 파우스트 메피스 토 텔레스 연기 ...
2240    해군 정신의학 장교 제롬 데이븐포트 덴젤 워싱턴 분 늘다 상관 폭행 하다 일 계급 ...
2241    어리다 시절 어머니 강요 트럼펫 불다 블릭 덴젤 워싱턴 은 성장하다 재능 있다 연 ...
2242    전편 여 주인공 애니 포터 산드라 블록 분 은 경찰 잭 헤어지다 지금 새 애인 알렉...
2243    TV 엔터테인먼트 뉴스 기자 맹활약 하다 자다 나가다 커리어 우먼 앨리슨 클럽 승진...
Name: 내용, Length: 2244, dtype: object


In [ ]:
#불용어 사전 만들기
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stop_words = set([x[0] for x in stopwords])

In [ ]:
def text_cleaning(text):
    hangul = re.compile('[^ a-z A-Z ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()
    words = okt.morphs(result, norm=True)
    words = [x for x in words if x not in stop_words]  # 불용어 제거
    return words

In [ ]:
book.내용 = book.내용.apply(lambda x : text_cleaning(x)) 

In [ ]:
print(book.내용)

0       [간, 묵묵하다, 물방울, 리다, 물방울, 작가, 사랑받다, 화가, 김창열, 침묵,...
1       [한국, 전쟁, 아직, 끝나다, 않다, 현재, 남, 북, 정치, 공작원, 상호, 침...
2       [중년, 여인, 무주, 시외버스, 터미널, 내, 린다, 그녀, 마을, 길, 지나, ...
3       [돈, 미치다, 필요하다, 태균, 돈, 들다, 자동차, 잃어버리다, 태민, 돈, 중...
4       [땅, 두르다, 싸다, 이웃, 다툼, 엄마, 딸, 기이하다, 관계, 낙향, 청년, ...
                              ...                        
2239    [차, 세계대전, 발발, 전, 나치, 점령, 독일, 연극, 파우스트, 메피스, 토,...
2240    [해군, 정신의학, 장교, 제롬, 데이븐포트, 덴젤, 워싱턴, 분, 늘다, 상관, ...
2241    [어리다, 시절, 어머니, 강요, 트럼펫, 불다, 블릭, 덴젤, 워싱턴, 은, 성장...
2242    [전편, 주인공, 애니, 포터, 산드라, 블록, 분, 은, 경찰, 잭, 헤어지다, ...
2243    [TV, 엔터테인먼트, 뉴스, 기자, 맹활약, 하다, 자다, 나가다, 커리어, 우먼...
Name: 내용, Length: 2244, dtype: object


In [ ]:
transformer = Countvectorizer()



---



In [ ]:
#숫자, 특수문자 제거 
def clean2(text):
  sent_clean = re.sub(r"[^a-zA-Z가-힣]"," ",text) #substraction 숫자, 특수문자 제거
  clean =[]

  return " ".join(clean)

In [ ]:
book.내용 = book.내용.apply(lambda x : clean2(x)) 

In [ ]:
print(book.내용)

0        
1        
2        
3        
4        
       ..
2239     
2240     
2241     
2242     
2243     
Name: 내용, Length: 2244, dtype: object


In [ ]:
book.head()

,Column1,제목,원제,제작연도,국가,장르,상영시간,연령등급,내용,리뷰
0,0,물방울을 그리는 남자,L'homme qui peint des gouttes d'eau,2020.0,"프랑스,한국",다큐멘터리,1시간 19분,전체 관람가,,"[[['이동진 평론가', '4.0', '삶이 어떻게 영화가 될 수 있는지 매우 아름..."
1,1,2차 송환,2차 송환,2022.0,한국,다큐멘터리,2시간 36분,12세 관람가,,"[[['이동진 평론가', '3.5', '여전히 뜻을 꺾지 않는 사람들을 다시 21년..."
2,2,달이 지는 밤,달이 지는 밤,2020.0,한국,모험,1시간 10분,15세 관람가,,"[[['이동진 평론가', '3.5', '꿈인 듯 현실인 듯 죽음인 듯 삶인 듯, 경..."
3,3,썬더버드,썬더버드,2021.0,한국,범죄/액션,1시간 37분,15세 관람가,,"[[['이동진 평론가', '3.0', '사북의 차가운 밤거리에서 실제로 엿보는 것 ..."
4,4,기기묘묘,기기묘묘,2022.0,한국,공포/액션/스릴러,1시간 56분,15세 관람가,,"[[['이동진 평론가', '2.5', '이탁 오민애 김재화. <불모지>가 참 좋았다..."


In [ ]:
#문장부호 제거 
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    return text.strip()

cleaned_train_corpus = []
cleaned_test_corpus = []
train.title = train.title.apply(lambda x : hanja.translate(x, 'substitution'))
test.title = test.title.apply(lambda x : hanja.translate(x, 'substitution'))

for sent in train['title']:
    cleaned_train_corpus.append(clean_punc(sent, punct, punct_mapping))
    
for sent in test['title']:
    cleaned_test_corpus.append(clean_punc(sent, punct, punct_mapping))


In [ ]:
#책 정보
item_transformers = [
    ('content_pipeline', Pipeline([
        ('content_extractor', TupleExtractor(0)),
        ('content_vectorizer', CountVectorizer(min_df = 2, ngram_range = (1,1)))
    ]))
    
]